## Importing Necessary Libraries

In [ ]:
!pip install onnxoptimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 79.8 MB/s eta 0:00:00


In [ ]:
!pip install onnx
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm.notebook import tqdm as tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision.models as models
import torchvision.datasets as datasets
import copy
import numpy as np
from tqdm.notebook import tqdm as tqdm
import time
from time import perf_counter
import timeit
import torch.nn.utils.prune as prune
import torch.quantization as quant
def timer(f,*args):   
    start = perf_counter()
    f(*args)
    return (1000 * (perf_counter() - start))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Preprocessing the data

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Load the CIFAR10 cifar10_dataset and apply the transformation pipeline
cifar10_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Calculating the sizes of the training, validation, and testing datasets
cifar10_train_size = int(0.7 * len(cifar10_dataset))
cifar10_val_size = int(0.1 * len(cifar10_dataset))
cifar10_test_size = len(cifar10_dataset) - cifar10_train_size - cifar10_val_size

# Splitting the cifar10_dataset into training, validation, and testing datasets
cifar10_train_dataset, cifar10_val_dataset, cifar10_test_dataset = torch.utils.data.random_split(cifar10_dataset, [cifar10_train_size, cifar10_val_size, cifar10_test_size])

batch_size = 32

# Creating data loaders for the training, validation, and testing datasets
cifar10_trainloader = torch.utils.data.DataLoader(cifar10_train_dataset, batch_size, shuffle=True, num_workers=2)
cifar10_valloader = torch.utils.data.DataLoader(cifar10_val_dataset, batch_size, shuffle=False, num_workers=2)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_test_dataset, batch_size, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 48527986.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Defining the data transformation pipeline
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Loading the svhn svhn_dataset and apply the transformation pipeline
svhn_dataset = torchvision.datasets.SVHN(root='./data',  download=True, transform=transform)

# Calculating the sizes of the training, validation, and testing datasets
svhn_train_size = int(0.7 * len(svhn_dataset))
svhn_val_size = int(0.1 * len(svhn_dataset))
svhn_test_size = len(svhn_dataset) - svhn_train_size - svhn_val_size

# Splitting the svhn_dataset into training, validation, and testing datasets
svhn_train_dataset, svhn_val_dataset, svhn_test_dataset = torch.utils.data.random_split(svhn_dataset, [svhn_train_size, svhn_val_size, svhn_test_size])

batch_size = 32

# Creating data loaders for the training, validation, and testing datasets
svhn_trainloader = torch.utils.data.DataLoader(svhn_train_dataset, batch_size, shuffle=True, num_workers=2)
svhn_valloader = torch.utils.data.DataLoader(svhn_val_dataset, batch_size, shuffle=False, num_workers=2)
svhn_testloader = torch.utils.data.DataLoader(svhn_test_dataset, batch_size, shuffle=False, num_workers=2)

100%|██████████| 182040794/182040794 [00:01<00:00, 92283337.37it/s] 


## Initializing the models

In [ ]:
vgg16 = models.vgg16(pretrained=True)
resnet50 = models.resnet50(pretrained=True)
resnet34 = models.resnet34(pretrained=True)
densenet121 = models.densenet121(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 66.2MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future.

In [ ]:
# Print the models' architectures
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Print the model's architecture
print(densenet121)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
# Print the model's architecture
print(resnet34)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Pruning

In [ ]:
import torch.nn.utils.prune as prune

def prune_model(model, prune_ratio):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=prune_ratio) # zeros out a portion of the weights based on their absolute magnitude
            prune.remove(module, 'weight')
        elif isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=prune_ratio) # zeros out a portion of the weights based on their absolute magnitude
            prune.remove(module, 'weight') #remove rued weights
    return model

In [ ]:
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import time
import timeit

In [ ]:
# Prune the model

vgg16_p = prune_model(vgg16, 0.2)  # prune 20% of weights


In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Evaluate the low-rank VGG16 model
correct = 0
total = 0
start = time.time()
with torch.no_grad():
    for images, labels in cifar10_testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16_p(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
end = time.time()

In [ ]:
t1 = end - start
t1

5.83979868888855

In [ ]:
# Evaluate the low-rank VGG16 model
correct = 0
total = 0
start = time.time()
with torch.no_grad():
    for images, labels in cifar10_testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
end = time.time()

In [ ]:
t2 = end - start
t2

6.519467115402222

In [ ]:
# Prune the model
resnet50_p = prune_model(resnet50, 0.2)  # prune 20% of weights

In [ ]:
# Prune the model
densenet121_p = prune_model(densenet121, 0.2)  # prune 20% of weights

In [ ]:
# Prune the model
resnet34_p = prune_model(resnet34, 0.2)  # prune 20% of weights

In [ ]:
print(vgg16)

In [ ]:
print(resnet50)

In [ ]:
print(densenet121)

In [ ]:
# Print the pruned model's architecture
print(resnet34)

# Quantization

In [ ]:
input_names = [ "actual_input" ]
output_names = [ "output" ]
resnet34.to(device)
densenet121.to(device)
resnet50.to(device)
vgg16.to(device)
dummy_input = torch.randn(1, 3, 224, 224)
dummy_input = dummy_input.to(device)

torch.onnx.export(vgg16,
                 dummy_input,
                 "vgg16.onnx",
                 verbose=False,
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True,
                 )

torch.onnx.export(densenet121,
                 dummy_input,
                 "densenet121.onnx",
                 verbose=False,
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True,
                 )

torch.onnx.export(resnet50,
                 dummy_input,
                 "resnet50.onnx",
                 verbose=False,
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True,
                 )

torch.onnx.export(resnet34,
                 dummy_input,
                 "resnet34.onnx",
                 verbose=False,
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True,
                 )

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
!python -m onnxoptimizer resnet34.onnx resnet34_opt.onnx
!python -m onnxoptimizer resnet50.onnx resnet50_opt.onnx
!python -m onnxoptimizer densenet121.onnx densenet121_opt.onnx
!python -m onnxoptimizer vgg16.onnx vgg16_opt.onnx

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def time_ort_model_evaluation(model_path):
    sess_options = onnxruntime.SessionOptions()
    sess_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    session = onnxruntime.InferenceSession(model_path, sess_options)

    time_per_inference = []
    for _ in range(10):
        dummy_input = torch.randn(1, 3, 224, 224)
        # compute ONNX Runtime output prediction
        ort_inputs = {session.get_inputs()[0].name: to_numpy(dummy_input)}
        start = perf_counter()
        session.run(None, ort_inputs)
        time_per_inference.append((1000 * (perf_counter() - start)))

    return np.mean(time_per_inference)

In [ ]:
import onnxruntime

In [ ]:
def quantize_onnx_model(onnx_model_path, quantized_model_path):
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnx
    onnx_opt_model = onnx.load(onnx_model_path)
    quantize_dynamic(onnx_model_path,
                     quantized_model_path,
                     weight_type=QuantType.QUInt8) #QInt8

    print(f"quantized model saved to:{quantized_model_path}")

quantize_onnx_model('resnet34_opt.onnx', 'resnet34_opt_quant.onnx')
quantize_onnx_model('densenet121_opt.onnx', 'densenet121_opt_quant.onnx')
quantize_onnx_model('resnet50_opt.onnx', 'resnet50_opt_quant.onnx')
quantize_onnx_model('vgg16_opt.onnx', 'vgg16_opt_quant.onnx')

quantized model saved to:resnet34_opt_quant.onnx
quantized model saved to:densenet121_opt_quant.onnx
quantized model saved to:resnet50_opt_quant.onnx
quantized model saved to:vgg16_opt_quant.onnx


### Size Comparison of ONNX and ONNX Quantized Models

In [ ]:
import os

In [ ]:
torch.save(resnet34.state_dict(), 'resnet34_model.pt')
resnet34.load_state_dict(torch.load('resnet34_model.pt'))

resnet34_size_bytes = os.path.getsize('resnet34_model.pt')
resnet34_size_mb = resnet34_size_bytes / (1024 * 1024)
print(f"Size of PyTorch Resnet-34 Model (MB): {resnet34_size_mb:.2f}")
print('ONNX full precision Resnet-34 model size (MB):', os.path.getsize("resnet34_opt.onnx")/(1024*1024))
print('ONNX quantized Resnet-34 model size (MB):', os.path.getsize("resnet34_opt_quant.onnx")/(1024*1024))

Size of PyTorch Resnet-34 Model (MB): 83.29
ONNX full precision Resnet-34 model size (MB): 83.13565731048584
ONNX quantized Resnet-34 model size (MB): 20.88004970550537


In [ ]:
torch.save(resnet50.state_dict(), 'resnet50_model.pt')
resnet50.load_state_dict(torch.load('resnet50_model.pt'))

resnet50_size_bytes = os.path.getsize('resnet50_model.pt')
resnet50_size_mb = resnet50_size_bytes / (1024 * 1024)
print(f"Size of PyTorch Resnet-50 Model (MB): {resnet50_size_mb:.2f}")
print('ONNX full precision Resnet-50 model size (MB):', os.path.getsize("resnet50_opt.onnx")/(1024*1024))
print('ONNX quantized Resnet-50 model size (MB):', os.path.getsize("resnet50_opt_quant.onnx")/(1024*1024))

Size of PyTorch Resnet-50 Model (MB): 97.80
ONNX full precision Resnet-50 model size (MB): 97.41436672210693
ONNX quantized Resnet-50 model size (MB): 24.532523155212402


In [ ]:
torch.save(vgg16.state_dict(), 'vgg16_model.pt')
vgg16.load_state_dict(torch.load('vgg16_model.pt'))

vgg16_size_bytes = os.path.getsize('vgg16_model.pt')
vgg16_size_mb = vgg16_size_bytes / (1024 * 1024)
print(f"Size of PyTorch VGG16 Model (MB): {vgg16_size_mb:.2f}")
print('ONNX full precision VGG16 model size (MB):', os.path.getsize("vgg16_opt.onnx")/(1024*1024))
print('ONNX quantized VGG16 model size (MB):', os.path.getsize("vgg16_opt_quant.onnx")/(1024*1024))

Size of PyTorch VGG16 Model (MB): 527.80
ONNX full precision VGG16 model size (MB): 527.7995767593384
ONNX quantized VGG16 model size (MB): 132.01754760742188


In [ ]:
torch.save(densenet121.state_dict(), 'densenet121_model.pt')
densenet121.load_state_dict(torch.load('densenet121_model.pt'))

densenet121_size_bytes = os.path.getsize('densenet121_model.pt')
densenet121_size_mb = densenet121_size_bytes / (1024 * 1024)
print(f"Size of PyTorch Densenet-121 Model (MB): {densenet121_size_mb:.2f}")
print('ONNX full precision Densenet-121 model size (MB):', os.path.getsize("densenet121_opt.onnx")/(1024*1024))
print('ONNX quantized Densenet-121 model size (MB):', os.path.getsize("densenet121_opt_quant.onnx")/(1024*1024))

Size of PyTorch Densenet-121 Model (MB): 31.02
ONNX full precision Densenet-121 model size (MB): 30.811330795288086
ONNX quantized Densenet-121 model size (MB): 8.444440841674805
